In [1]:
import sys
sys.path.append('/home/koshirshov/koshirshov/RecBot')
import config
from pandas_utils import load_data
from models import PopularRecommender, LightFmModel
from recommendation_utils import create_recommendations_from_one_model, create_submission_file
from metrics import compute_metrics
import pandas as pd

In [2]:
path = config.proceed_data_path
users_df = load_data(path + 'users_processed.pkl',)
items_df = load_data(path + 'items_processed.pkl',)
train_interactions = load_data(path + 'train_interactions.pkl',)
test_interactions = load_data(path + 'test_interactions.pkl',)
sample_submission = load_data(path + 'sample_submission_processed.pkl',)

In [3]:
#popular
pop_model = PopularRecommender(days=30)
pop_model.fit(train_interactions)
df_recommendations = create_recommendations_from_one_model(pop_model, test_interactions['user_id'].unique())
metrics = compute_metrics(train_interactions, test_interactions, df_recommendations)
print(metrics)

{'Precision@1': 0.07489781772115592, 'Recall@1': 0.04826541257304038, 'Precision@3': 0.07616463895594809, 'Recall@3': 0.1375204993719853, 'Precision@10': 0.03765387097545237, 'Recall@10': 0.2108414786587126, 'MAP@10': 0.10311451737903853, 'Novelty@10': 3.435862548188481}


In [4]:
#popular + light_fm
light_fm_model = LightFmModel()
light_fm_model.prepare_csr_matrix(train_interactions)

users_with_interactions_in_train = (set(train_interactions['user_id'])&set(test_interactions['user_id'])) - light_fm_model.users_not_in_matrix
users_without_interactions_in_train = (set(test_interactions['user_id']) - users_with_interactions_in_train).union(light_fm_model.users_not_in_matrix)
users_with_interactions_in_train = [user for user in users_with_interactions_in_train]
users_without_interactions_in_train = [user for user in users_without_interactions_in_train]

light_fm_model.fit(30)
df_recommendations_lfm = create_recommendations_from_one_model(light_fm_model, users_with_interactions_in_train)
df_recommendations_popular = create_recommendations_from_one_model(pop_model, users_without_interactions_in_train)
df_recommendations = pd.concat([df_recommendations_lfm, df_recommendations_popular], axis = 0)
metrics = compute_metrics(train_interactions, test_interactions, df_recommendations)
print(metrics)

predict from light_fm model, users:: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:47<00:00,  6.75s/it]


{'Precision@1': 0.08078375600544972, 'Recall@1': 0.05083565033208718, 'Precision@3': 0.07590768936587232, 'Recall@3': 0.1362939349624927, 'Precision@10': 0.03927623873604704, 'Recall@10': 0.21595715683431518, 'MAP@10': 0.10531653632656372, 'Novelty@10': 3.491949849728626}


In [5]:
{'Precision@1': 0.08462604871286182, 'Recall@1': 0.05317130005803583, 'Precision@3': 0.07436200811084288, 'Recall@3': 0.13359372126257665, 'Precision@10': 0.03933181155436575, 'Recall@10': 0.21642830347395992, 'MAP@10': 0.10625493586122989, 'Novelty@10': 3.67470572866023}

{'Precision@1': 0.08462604871286182,
 'Recall@1': 0.05317130005803583,
 'Precision@3': 0.07436200811084288,
 'Recall@3': 0.13359372126257665,
 'Precision@10': 0.03933181155436575,
 'Recall@10': 0.21642830347395992,
 'MAP@10': 0.10625493586122989,
 'Novelty@10': 3.67470572866023}